In [11]:
%create_livy_session \
--cluster ephemeral-cluster \
--id ses1 \
--conf spark.jars.packages=io.delta:delta-core_2.12:0.8.0 \
--conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension \
--conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog \
--conf spark.sql.hive.metastore.sharedPrefixes=com.amazonaws,ru.yandex.cloud \
--conf spark.sql.warehouse.dir=s3a://keshaaa/wh

Starting session ses1


In [88]:
#!spark --cluster ephemeral-cluster --session ses1
S3_BUCKET_NAME="keshaaa"
inputDF = spark.createDataFrame(
    [
        ("100", "2015-01-01", "2015-01-01T13:51:39.340396Z"),
        ("101", "2015-01-01", "2015-01-01T12:14:58.597216Z"),
        ("102", "2015-01-01", "2015-01-01T13:51:40.417052Z"),
        ("103", "2015-01-01", "2015-01-01T13:51:40.519832Z"),
        ("104", "2015-01-02", "2015-01-01T12:15:00.512679Z"),
        ("105", "2015-01-02", "2015-01-01T13:51:42.248818Z"),
    ],
    ["id", "creation_date", "last_update_time"],
)

# Write a DataFrame as a Delta dataset
inputDF.write.format("delta").mode("overwrite").option(
    "overwriteSchema", "true"
).partitionBy("creation_date").save(f"s3a://{S3_BUCKET_NAME}/tmp/delta/")

In [82]:
%%sh

ls -lR /home/jupyter/datasphere/s3/s3fs/tmp/delta | tee /tmp/delta_op_001

/home/jupyter/datasphere/s3/s3fs/tmp/delta:
total 0


In [89]:
#!spark --cluster ephemeral-cluster --session ses1
from pyspark.sql.functions import lit

# Create a new DataFrame from the first row of inputDF with a different creation_date value
updateDF = inputDF.where("id = 100").withColumn("creation_date", lit("2022-01-11"))

from delta.tables import *
from pyspark.sql.functions import *

deltaTable = DeltaTable.forPath(spark, f"s3a://{S3_BUCKET_NAME}/tmp/delta/")

deltaTable.alias("oldData") \
  .merge(
    updateDF.alias("newData"),
    "oldData.id = newData.id") \
  .whenMatchedUpdate(set = { "creation_date": col("newData.creation_date") }) \
  .execute()

deltaTable.toDF().show()

+---+-------------+--------------------+
| id|creation_date|    last_update_time|
+---+-------------+--------------------+
|100|   2022-01-11|2015-01-01T13:51:...|
|103|   2015-01-01|2015-01-01T13:51:...|
|101|   2015-01-01|2015-01-01T12:14:...|
|102|   2015-01-01|2015-01-01T13:51:...|
|104|   2015-01-02|2015-01-01T12:15:...|
|105|   2015-01-02|2015-01-01T13:51:...|
+---+-------------+--------------------+


In [90]:
#!spark --cluster ephemeral-cluster --session ses1
inputDF.show()

+---+-------------+--------------------+
| id|creation_date|    last_update_time|
+---+-------------+--------------------+
|100|   2015-01-01|2015-01-01T13:51:...|
|101|   2015-01-01|2015-01-01T12:14:...|
|102|   2015-01-01|2015-01-01T13:51:...|
|103|   2015-01-01|2015-01-01T13:51:...|
|104|   2015-01-02|2015-01-01T12:15:...|
|105|   2015-01-02|2015-01-01T13:51:...|
+---+-------------+--------------------+


In [97]:
#!spark --cluster ephemeral-cluster --session ses1
deltaTable.history().show()

+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|version|          timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|
+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|      1|2023-07-03 16:55:00|  null|    null|    MERGE|[predicate -> (ol...|null|    null|     null|          0|          null|        false|[numTargetRowsCop...|        null|
|      0|2023-07-03 16:54:28|  null|    null|    WRITE|[mode -> Overwrit...|null|    null|     null|       null|          null|        false|[numFiles -> 3, n...|        null|
+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------

In [102]:
#!spark --cluster ephemeral-cluster --session ses1
history = spark.sql(f"DESCRIBE HISTORY delta.`s3a://{S3_BUCKET_NAME}/tmp/delta/`")
history.show()

latest_version = history.selectExpr("max(version)").collect()[0][0]
print(f"Latest version is: {latest_version}")

+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|version|          timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|
+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|      1|2023-07-03 16:55:00|  null|    null|    MERGE|[predicate -> (ol...|null|    null|     null|          0|          null|        false|[numTargetRowsCop...|        null|
|      0|2023-07-03 16:54:28|  null|    null|    WRITE|[mode -> Overwrit...|null|    null|     null|       null|          null|        false|[numFiles -> 3, n...|        null|
+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------

In [108]:
#!spark --cluster ephemeral-cluster --session ses1
df2 = (
    spark.read.format("delta")
    .option("versionAsOf", 0)
    .load(f"s3a://{S3_BUCKET_NAME}/tmp/delta/")
)
df2.sort("id").show()
df2 = (
    spark.read.format("delta")
    .option("versionAsOf", 1)
    .load(f"s3a://{S3_BUCKET_NAME}/tmp/delta/")
)
df2.sort("id").show()

+---+-------------+--------------------+
| id|creation_date|    last_update_time|
+---+-------------+--------------------+
|100|   2015-01-01|2015-01-01T13:51:...|
|101|   2015-01-01|2015-01-01T12:14:...|
|102|   2015-01-01|2015-01-01T13:51:...|
|103|   2015-01-01|2015-01-01T13:51:...|
|104|   2015-01-02|2015-01-01T12:15:...|
|105|   2015-01-02|2015-01-01T13:51:...|
+---+-------------+--------------------+

+---+-------------+--------------------+
| id|creation_date|    last_update_time|
+---+-------------+--------------------+
|100|   2022-01-11|2015-01-01T13:51:...|
|101|   2015-01-01|2015-01-01T12:14:...|
|102|   2015-01-01|2015-01-01T13:51:...|
|103|   2015-01-01|2015-01-01T13:51:...|
|104|   2015-01-02|2015-01-01T12:15:...|
|105|   2015-01-02|2015-01-01T13:51:...|
+---+-------------+--------------------+


In [109]:
#!spark --cluster ephemeral-cluster --session ses1
spark.sql(f"VACUUM delta.`s3a://{S3_BUCKET_NAME}/tmp/delta/`")

DataFrame[path: string]


In [101]:
%%bash

ls -lR /home/jupyter/datasphere/s3/s3fs/tmp/delta | tee /tmp/delta_op_002

/home/jupyter/datasphere/s3/s3fs/tmp/delta:
total 2
drwxrwxrwx 1 root root 0 Jan  1  1970 _delta_log
drwxrwxrwx 1 root root 0 Jan  1  1970 creation_date=2015-01-01
drwxrwxrwx 1 root root 0 Jan  1  1970 creation_date=2015-01-02
drwxrwxrwx 1 root root 0 Jan  1  1970 creation_date=2022-01-11

/home/jupyter/datasphere/s3/s3fs/tmp/delta/_delta_log:
total 3
-rwxrwxrwx 1 root root 1451 Jul  3 16:54 00000000000000000000.json
-rwxrwxrwx 1 root root 1461 Jul  3 16:55 00000000000000000001.json

/home/jupyter/datasphere/s3/s3fs/tmp/delta/creation_date=2015-01-01:
total 4
-rwxrwxrwx 1 root root 793 Jul  3 16:54 part-00000-5e6ff72f-54e3-4f5f-8c18-ec203ff86455.c000.snappy.parquet
-rwxrwxrwx 1 root root 875 Jul  3 16:54 part-00001-ae426758-da19-46cf-9e9b-b4b1fdd2653d.c000.snappy.parquet
-rwxrwxrwx 1 root root 875 Jul  3 16:54 part-00011-475a45c6-2216-4362-a031-70e9518a46ec.c000.snappy.parquet
-rwxrwxrwx 1 root root 875 Jul  3 16:54 part-00088-ed714df7-40f6-4198-876c-cd424c50cdb0.c000.snappy.parquet

/

In [ ]:
%delete_livy_session \
--cluster ephemeral-cluster \
--id ses1